In [11]:
pwd

'/home/catherine/code/elsebasmar/london-bss/londonbss/notebooks/data_collection'

In [12]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd


In [13]:
df_test = pd.read_csv("../../../raw_data/final_features_df.csv")

In [16]:
df_test.head()

,Unnamed: 0,temperature,rainfall,snowfalll,cloudcover,wind_speed,wind_direction,date,sunrise,sunset,...,event_title,event_start_date,event_end_date,event_location,event_latitude,event_longitude,elisabeth_line,lockdown,school_holidays,strike
0,0,12.5,0.1,0.0,100,6.9,189,2022-01-02,2022-01-02T09:06,2022-01-02T17:03,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,christmas,NaN
1,1,12.4,0.1,0.0,100,20.2,239,2022-01-02,2022-01-02T09:06,2022-01-02T17:03,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,christmas,NaN
2,2,12.4,0.0,0.0,100,19.0,241,2022-01-02,2022-01-02T09:06,2022-01-02T17:03,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,christmas,NaN
3,3,12.4,0.0,0.0,100,17.4,248,2022-01-02,2022-01-02T09:06,2022-01-02T17:03,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,christmas,NaN
4,4,12.4,0.0,0.0,100,17.4,248,2022-01-02,2022-01-02T09:06,2022-01-02T17:03,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,christmas,NaN


In [17]:
df_test.shape

(9192, 22)

In [23]:
X_train = df_test[0:7000, :]

InvalidIndexError: (slice(0, 7000, None), slice(None, None, None))

In [15]:
# ENCODE EVENTS

##London_zones --> Should that go into params ?? if we use them somewhere else?
# South East limits
se_limit_lat = 51.523811 # everthing lower than this value is south east
se_limit_lon = -0.101342 # everything greater than this value is south east

# East limits
e_limit_lat = 51.523811 # everything greater than this value is east
e_limit_lon = -0.022306 # everything greater than this is east

# South West limits
sw_limit_lat = 51.501631 # everthing lower than this value is south east
sw_limit_lon = -0.101342 # everything lower than this value is south east

# West limits
w_limit_lat = 51.501072  # everthing greater than this value is south east
w_limit_lon = -0.182406  # everthing lower than this value is south east

# Central limits
c_toplimit_lat = 51.501072 # everthing greater than this value is central
c_lowlimit_lat = 51.523811 # everything lower than this value is central

c_toplimit_lon = -0.101342 # everything lower than this value is central
c_lowlimit_lon = -0.182406 # everthing greater than this value is central

# North limits
n_toplimit_lon = -0.022306 # everything lower than this value is central
n_lowlimit_lon = -0.182406 # everthing greater than this value is central

n_lowlimit_lat = 51.523811 # everthing greater than this value is central

## Encode zone based on events lat and long
def encode_events_zone(X: pd.DataFrame):
    london_loc = []
    for i in range(len(X.index)):
        if X['event_latitude'][i] == "London-wide":
            london_loc.append('London_all') # London wide
        elif X['event_latitude'][i] > c_toplimit_lat and X['event_latitude'][i]< c_lowlimit_lat and X['event_longitude'][i]< c_toplimit_lon and X['event_longitude'][i]>c_lowlimit_lon:
                london_loc.append('Central') # Central
        elif X['event_longitude'][i] < n_toplimit_lon and X['event_longitude'][i] > n_lowlimit_lon and X['event_latitude'][i] > n_lowlimit_lat:
            london_loc.append('North') # North
        elif X['event_longitude'][i] < w_limit_lon and X['event_latitude'][i] > w_limit_lat:
            london_loc.append('West') # West
        elif X['event_longitude'][i] < sw_limit_lon and X['event_latitude'][i] < sw_limit_lat:
            london_loc.append('South_West')  # South_West
        elif X['event_longitude'][i] > e_limit_lon and X['event_latitude'][i] > e_limit_lat:
            london_loc.append('East') # East
        elif X['event_longitude'][i] > se_limit_lon and X['event_latitude'][i] < se_limit_lat:
            london_loc.append('South_East') # South East
        else:
            london_loc.append('Other')

    X["London_zone"] = london_loc

    return X

    ## OHE for London zone
def London_zone_manual_ohe(X: pd.DataFrame):
    X["London_zone_Central"] = X.apply(lambda x: 1.0 if x['London_zone'] == "Central" else x = 0.0, axis=1)
    X["London_zone_North"] = X.apply(lambda x: 1.0 if x['London_zone'] == "North" else x = 0.0, axis=1)
    X["London_zone_West"] = X.apply(lambda x: 1.0 if x['London_zone'] == "West" else x = 0.0, axis=1)
    X["London_zone_South_West"] = X.apply(lambda x: 1.0 if x['London_zone'] == "South_West" else x = 0.0, axis=1)
    X["London_zone_South_East"] = X.apply(lambda x: 1.0 if x['London_zone'] == "South_East" else x = 0.0, axis=1)
    X["London_zone_East"] = X.apply(lambda x: 1.0 if x['London_zone'] == "East" else x = 0.0, axis=1)
    X["London_zone_London_all"] = X.apply(lambda x: 1.0 if x['London_zone'] == "London_all" else x = 0.0, axis=1)
    return X

def london_all_encoding(X: pd.DataFrame):
    X["London_zone_Central"] = X.apply(lambda x: 1.0 if x['London_all'] == 1.0 else x['London_zone_Central'], axis=1)
    X["London_zone_North"] = X.apply(lambda x: 1.0 if x['London_all'] == 1.0 else x['London_zone_North'], axis=1)
    X["London_zone_West"] = X.apply(lambda x: 1.0 if x['London_all'] == 1.0 else x['London_zone_West'], axis=1)
    X["London_zone_South_West"] = X.apply(lambda x: 1.0 if x['London_all'] == 1.0 else x['London_zone_South_West'], axis=1)
    X["London_zone_South_East"] = X.apply(lambda x: 1.0 if x['London_all'] == 1.0 else x['London_zone_South_East'], axis=1)
    X["London_zone_East"] = X.apply(lambda x: 1.0 if x['London_all'] == 1.0 else x['London_zone_East'], axis=1)
    X.drop(columns= ["London_all"], inplace = True)
    return X

    ## Encode event_title
def encoding_strings(text):
    if text == 0:
        pass
    else:
        text = 1
    return text

def encode_event_title(X: pd.DataFrame):
    X['event_title'] = X['event_title'].fillna(0)
    X['event_title'] = X['event_title'].apply(encoding_strings)
    X.drop(columns = ["event_title"], inplace = True)
    X = X.groupby('timestamp').sum()
    return X

events_pipeline = make_pipeline(
        FunctionTransformer(encode_events_zone),
        FunctionTransformer(London_zone_manual_ohe),
        FunctionTransformer(london_all_encoding),
        FunctionTransformer(encode_event_title),
        remainder = "passthrough"
)

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (4257615174.py, line 60)

In [ ]:
# ENCODE OTHER COLUMNS
    ## Encode elisabeth line and lockdown
def bool_to_int(X: pd.DataFrame):
    X['elisabeth_line'] = np.where(X['elisabeth_line'], '1', '0')
    X['lockdown'] = np.where(X['lockdown'], '1', '0')
    return X

    ## Encode strikes and school holidays
def encode_strikes_holidays(X: pd.DataFrame):
    X['strike'] = X['strike'].fillna(0)
    X['strike'] = X['strike'].apply(encoding_strings)
    X['school_holidays'] = X['school_holidays'].fillna(0)
    X['school_holidays'] = X['school_holidays'].apply(encoding_strings)
    return X

    ## Add additional dates details
def add_details_date(X: pd.DataFrame):
    X["year"] = X["timestamp"].str[:4]
    X["month"] = X["timestamp"].str[6:7]
    X["day"] = X["timestamp"].str[9:10]
    X["hour"] = X["timestamp"].str[12:13]
    X["weekday"] = X["timestamp"].dt.weekday
    return X

def encode_day_nighttime(X: pd.DataFrame):
    X["daytime"] = X["daytime"].replace("daytime", "1")
    X["daytime"] = X["daytime"].replace("nighttime", "0")
    return X

additional_pipeline = make_pipeline(
        FunctionTransformer(bool_to__int),
        FunctionTransformer(encore_strikes_holidays),
        FunctionTransformer(add_details_date),
        FunctionTransformer(encode_day_nighttime),
        remainder = "passthrough"
)

In [ ]:
# ENCODE WEATHER COLUMNS
weather_pipeline = make_pipeline(MinMaxScaler()
    ["temperature", "rainfall", "snowfall", "cloudcover", "wind_speed", "wind_direction"]
    )

In [ ]:
# FINAL FULL PREPROCESSOR
final_preprocessor = ColumnTransformer(
    [
        ("events_encoding", events_pipeline),
        ("weather_encoding", weather_pipeline),
        ("additional_categories", additional_pipeline),
    ],
        n_jobs=-1,
)
    return final_preprocessor

In [ ]:
# FIT
def fit_transform_features(X: pd.DataFrame):
    if X == "X_train":
        X_train_processed = final_preprocessor.fit_transform(X)
    elif X == "X_val":
        X_val_processed = final_preprocessor.transform(X)
    else:
        X_test_processed = final_preprocessor.transform(X)

    return X

In [13]:
fit_transform_features(X_train)

In [ ]:
# DROP COLUMNS